In [ ]:
from IBEAforBioemus.ibea import IBEA
import numpy as np 
from IBEAforBioemus.objectives_computation_lvr import evaluate_objectives, clear_files, compute_goal_distr
import scipy.io
import warnings 
import time
''' 5 variables in this case are (in order): inh/exc ratio, max prob of connection, AMPA/NMDA ratio, NMDA syn weight, AMPA syn weight, synaptic noise.'''

warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

clear_files("/home/ubuntu/bioemus/data", ".bin")
clear_files("/home/ubuntu/bioemus/config", ".json")
clear_files("/home/ubuntu/bioemus/config", ".txt")

alpha = 100
fun = evaluate_objectives
lbound = np.array([0.05, 0.05, 0.1, 0.05, 0.05, 0.01])
ubound = np.array([0.9, 0.5, 1, 1, 1, 0.1])
remaining_budget = np.inf
n_offspring = 15
max_generations = 50
var = (ubound-lbound)/7

ISI_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/ISI.mat')
MFR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/MFR.mat')
LvR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/lvr.mat')
MBR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/MBR.mat')
ISI_goal = np.sum(ISI_dict['ISI'], axis=0)
ISI_goal /=np.sum(ISI_goal)
MFR_goal = MFR_dict['mfr']
LvR_goal = LvR_dict['lvr']
MBR_goal = MBR_dict['mbr']
goal = [MFR_goal, MBR_goal, LvR_goal, ISI_goal]

goal_distr = compute_goal_distr(goal)

solution = IBEA(kappa=0.05, alpha = alpha, n_offspring= n_offspring, max_generations= max_generations, seedit = 333, var=var, n_sbx=2)
start = time.time()
index, best_parameters, distances, true_distr = solution.ibea(fun, lbound, ubound, remaining_budget, goal_distr)
print(f"Best configuration is the number {index} with parameters {best_parameters} and distances {distances}")
end = time.time()
print(end-start)

In [ ]:
import matplotlib.pyplot as plt
import scipy
from IBEAforBioemus.objectives_computation_lvr import do_analysis, compute_objectives, compute_goal_distr
import numpy as np

CONFIG_NAME = "genesis3"
filename = "/home/ubuntu/bioemus/data/raster_" + CONFIG_NAME + ".bin"

RASTER_LIST = CONFIG_NAME
password = 'kriachan'
ISI_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/ISI.mat')
MFR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/MFR.mat')
LvR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/lvr.mat')
MBR_dict = scipy.io.loadmat('/home/ubuntu/bioemus/IBEAforBioemus/Goals/MBR.mat')
ISI_goal = ISI_dict['ISI'][3]
MFR_goal = MFR_dict['mfr']
LvR_goal = LvR_dict['lvr']
MBR_goal = MBR_dict['mbr']
goal = [MFR_goal, MBR_goal, LvR_goal, ISI_goal]
goal_distr = compute_goal_distr(goal)

individual_metrics = do_analysis(filename, RASTER_LIST, [0.37026566, 0.07541495, 0.3370662, 0.34873528, 0.04206774])
distances, true_distributions = compute_objectives(individual_metrics, goal_distr)
print(distances)

fig, axs = plt.subplots(2, 2, figsize=(10, 10))  # Adjust figsize as needed
axs = axs.flatten()
names = ['MFR', 'MBR', 'LvR', 'ISI']
ISI_bin_edges = np.linspace(0,100, 200)
ISI_bin_centers = (ISI_bin_edges[:-1] + ISI_bin_edges[1:]) / 2
x1 = [np.linspace(0.2, 100, 199), np.linspace(0.2, 100, 199), np.linspace(0.2, 2, 199), ISI_bin_centers]
for i in range(len(true_distributions)):
    if i == 2:
        width = 0.02
    else:
        width = 1
    axs[i].bar(x1[i], true_distributions[i], width, color='blue', alpha=0.6, hatch='\\', label = 'Data')
    axs[i].bar(x1[i], goal_distr[i], width, color='red', alpha=0.6, hatch='\\', label = 'Goal')
    axs[i].legend()
    axs[i].set_title(f'{names[i]}')
    axs[i].grid(True)
plt.tight_layout()
plt.show()